<a href="https://colab.research.google.com/github/plaban1981/Fastai/blob/master/Copy_of_meme_saheb_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Drive connectivity

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setting up

In this section, we'll perform all operations that we require *before* we get to scraping

---

In [2]:
from bs4 import BeautifulSoup
import requests
import shutil
import os

print('Modules imported')

Modules imported


We're going to scrape our memes off of memegenerator.net  
Let's start by setting up some globsl variables

In [7]:
!mkdir '/content/drive/My Drive/dankLearning'

mkdir: cannot create directory ‘/content/drive/My Drive/dankLearning’: File exists


In [4]:
saveFolder = '/content/drive/My Drive/dankLearning'
if not os.path.exists(saveFolder):
  os.mkdir(saveFolder)

trainDir = os.path.join(saveFolder, 'seg_train')
trainPath = os.path.join(trainDir, 'seg_train')
testDir = os.path.join(saveFolder, 'seg_test')
testPath = os.path.join(testDir, 'seg_test')
predDir = os.path.join(saveFolder, 'seg_pred')
predPath = os.path.join(predDir, 'seg_pred')
saveDirs = [trainDir, testDir, predDir]
savePaths = [trainPath, testPath, predPath]
for pth in saveDirs:
  if not os.path.exists(pth):
    os.mkdir(pth)
for pth in savePaths:
  if not os.path.exists(pth):
    os.mkdir(pth)

noPageTemplates = 1  # the number of pages of templates we want from the site
# the number of pages of captions we want for each template
noPageCaptions = [2, 1, 1] # train, test, pred

siteUrl = 'https://memegenerator.net'
pageModifier = '/memes/popular/alltime/page/' # need to add this to the URL to direct it to a specific page

print('Set up successfully')

Set up successfully


# Getting URLs

This is the part where we find the URLs to all the meme templates we want.

Meme Generator organizes memes by template and each template, therefore, has a separate page with multiple memes using different captions.

---

In [5]:
memeURLs = list() # a list to store links to all the meme pages
 
memeIden = 'generator-line' # this is the class that this site uses to define the list elements in their memes
 
for pg in range(noPageTemplates):
  # page numbers begin with 1 while this loop goes from 0 to noPageTemplate-1
  req = requests.get(siteUrl + pageModifier + str(pg + 1))
  soup = BeautifulSoup(req.text, 'html.parser')
  elements = soup.find_all(class_ = memeIden)
  memeURLs += elements
 
print(memeURLs[0])
print() # empty line
print('Count of meme templates:', len(memeURLs))

<div class="generator-line">
<div class="right mt40px">
<div class="voter arrows" data-entity-id="2" data-entity-name="Generator" onclick="alertLoginRequired();">
<div class="downvote" onmousedown="if (event.button != 0) return; Voter_OnDownVoteClick(this.parentNode);" title="Downvote">
</div>
<div class="score">
            -16,940
        </div>
<div class="upvote" onmousedown="if (event.button != 0) return; Voter_OnUpVoteClick(this.parentNode);" title="Upvote">
</div>
</div>
</div>
<a href="/Y-U-No">
<img class="mr10px loading-bg" src="https://memegenerator.net/img/images/120x120/166088.jpg">
<div class="info">
<span class="display-name">Y U No</span>
<br/>
<span class="instances-count bold black">
<span>2M</span> images
            </span>
</div>
</img></a>
</div>

Count of meme templates: 15


# Collecting memes

In this section, we'll finally download all the memes and captions to our drives

---

We need to save the image with the name found with class - *display-name*

Also, we need to go to the meme's page and get captions of various versions of it. We'll store all captions in the same file. Let's make that file now.

In [10]:
!sudo python3 -m pip install "requests[security]"

     |████████████████████████████████| 61kB 1.7MB/s 
     |████████████████████████████████| 2.6MB 6.7MB/s 


In [12]:
from requests.exceptions import ConnectionError

In [14]:
capFileName = 'captions.txt'
capFilePath = os.path.join(saveFolder, capFileName)
# 'a' mode opens a file without altering data but it also creates it if it doesn't exist
with open(capFilePath, 'a') as capFile:
  pass

captionIden = 'generator-img'
memePgMod = '/images/popular/alltime/page/' # page modifier for a specific template's pages
for count, meme in enumerate(memeURLs):
  memeName = meme.find(class_ = 'display-name').text.strip()
  # memePath = os.path.join(saveFolder, memeName)
  # if not os.path.exists(memePath):
  #   os.mkdir(memePath)

  # ideally, we should grab the extension from the image but we'll let it be this once
  imgName = memeName + '.jpg'
  imgPath = os.path.join(saveFolder, imgName)
  imgUrl = meme.find('img')['src']
  res = requests.get(imgUrl, stream=True)
  try:
    with open(imgPath, 'wb') as outFile:
      shutil.copyfileobj(res.raw, outFile)
  except UnicodeEncodeError:
    continue
  except FileNotFoundError:
    continue
  print(count, ') ', imgName, 'captured')

  memePaths = [os.path.join(p, memeName) for p in savePaths]
  memePaths[-1] = savePaths[-1] # Pred doesn't need class labels
  for mp in memePaths:
    if not os.path.exists(mp):
      os.mkdir(mp)

  # moving to the template page
  pageURL = siteUrl + meme.find('a')['href']
  # for train set
  for i in range(3):  # train, test, pred
    capCount = 0
    pageNum = 1
    for pg in range(noPageCaptions[i]):
      pageNum += pg
      req = requests.get(pageURL + memePgMod + str(pageNum))
      soup = BeautifulSoup(req.text, 'html.parser')
      memes = soup.find_all(class_ = captionIden)
      capCount += len(memes)
      for meme in memes:
        caption = meme.find('a')['href']
        caption = caption[19:]  # removing the start of the URL
        caption = ' '.join(caption.split('-'))  # replacing hyphens with spaces
        caption = caption.strip() # removing extra whitespace
        with open(capFilePath, 'a') as capFile:
          capFile.write(caption + '\n')
        captionImage = meme.find('img')['src']
        capImgName = '-'.join(caption.split()) + '.jpg'
        capImgPath = os.path.join(memePaths[i], capImgName)
        res = requests.get(captionImage, stream=True)
        try:
          with open(capImgPath, 'wb') as outFile:
            shutil.copyfileobj(res.raw, outFile)
        except UnicodeEncodeError:
          continue
        except FileNotFoundError:
          continue
        except ConnectionError as err:
          if nb_tries == 0:
            raise err
          else:
            time.sleep(5)

    print('Count of captions:', capCount)

0 )  Y U No.jpg captured
Count of captions: 30
Count of captions: 15
Count of captions: 15
1 )  Bad Luck Brian.jpg captured


ConnectionError: ignored